<a href="https://colab.research.google.com/github/wigneshh/NLP/blob/main/Assignment_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Example text data (you can replace this with any larger corpus) text = """ Once upon a time, there was a little girl named Red Riding Hood. She loved to visit her grandmother, who lived in the woods. One day, her mother asked her to take a basket of goodies to her grandmother. On her way through the woods, she met a big bad wolf who wanted to eat her. [CO5]
(i) Build the Transformer Model on above dataset

(ii) Train the model using 20, 60, 70 epochs

 (iii) After training, use the model to generate new text by feeding it an initial seed text

(iv) Experimenting and Improving the Model by large dataset and hyper tune parameter.

In [14]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, MultiHeadAttention, LayerNormalization, Dense, Dropout
from tensorflow.keras import Sequential, Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

text = """Once upon a time, there was a little girl named Red Riding Hood. She loved to visit her grandmother, who lived in the woods. One day, her mother asked her to take a basket of goodies to her grandmother. On her way through the woods, she met a big bad wolf who wanted to eat her."""

vocab_size = 1000
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts([text])


sequences = tokenizer.texts_to_sequences([text])[0]


input_sequences = []
for i in range(1, len(sequences)):
    input_sequences.append(sequences[:i+1])


max_sequence_len = max(len(x) for x in input_sequences)
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')

X, y = input_sequences[:,:-1], input_sequences[:,-1]
y = tf.keras.utils.to_categorical(y, num_classes=vocab_size)


class TransformerModel(Model):
    def __init__(self, vocab_size, embed_dim, num_heads, ff_dim):
        super(TransformerModel, self).__init__()
        self.embedding = Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.attention = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = Sequential([Dense(ff_dim, activation="relu"), Dense(embed_dim)])
        self.layernorm1 = LayerNormalization()
        self.layernorm2 = LayerNormalization()
        self.dropout1 = Dropout(0.1)
        self.dropout2 = Dropout(0.1)
        self.final_layer = Dense(vocab_size, activation="softmax")

    def call(self, inputs):
        x = self.embedding(inputs)
        attn_output = self.attention(x, x)
        x = self.layernorm1(x + self.dropout1(attn_output))
        ffn_output = self.dense_proj(x)
        x = self.layernorm2(x + self.dropout2(ffn_output))
        return self.final_layer(x[:, -1])


embed_dim = 128
num_heads = 4
ff_dim = 512

transformer_model = TransformerModel(vocab_size, embed_dim, num_heads, ff_dim)
transformer_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])


epochs_list = [20,60,70]
for epochs in epochs_list:
    print(f"\nTraining model with {epochs} epochs:")
    transformer_model.fit(X, y, epochs=epochs, verbose=1)


def generate_text(model, tokenizer, seed_text, max_length=50):
    for _ in range(max_length):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len - 1, padding='pre')
        predicted = model.predict(token_list, verbose=0)
        predicted_word_index = np.argmax(predicted, axis=-1).item()
        output_word = tokenizer.index_word.get(predicted_word_index, "")
        seed_text += " " + output_word
    return seed_text


seed_text = "Once upon a time"
print("\nGenerated text:", generate_text(transformer_model, tokenizer, seed_text))





Training model with 20 epochs:
Epoch 1/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 148ms/step - accuracy: 0.0000e+00 - loss: 7.0221
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 143ms/step - accuracy: 0.2288 - loss: 5.9813
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 137ms/step - accuracy: 0.2271 - loss: 5.3617
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 149ms/step - accuracy: 0.3085 - loss: 4.8367
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 164ms/step - accuracy: 0.4956 - loss: 4.2609
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 141ms/step - accuracy: 0.6239 - loss: 3.8384
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 139ms/step - accuracy: 0.7487 - loss: 3.3363
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step - accuracy: 0.7608 - loss: 2.9252
Epoch 9/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step - accuracy: 0.7816 - loss: 2.4931
Epoch 10/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step - accuracy: 0.7816 - loss: 2.1712
Epoch 11/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 144ms/step - accuracy: 0.8025 - loss: 1.8503
Epoch 12/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1